In [1]:
import time
import requests
import pandas as pd

# Рейтинг интерфакса

In [2]:
def get_interfax_list_by_year(year, page=1):
    url = f"https://academia.interfax.ru/data/rating/?rating=1&year={year}&page={page}"
    result = []
    resp = requests.get(url)

    # Бережём источник данных
    time.sleep(1)

    if resp.status_code == 200:
        json_data = resp.json()
        result += json_data['universities']
        page_count = json_data['page_count']

        if page != page_count:
            result += get_interfax_list_by_year(year, page + 1)
    else:
        print(f"{url} return status code {resp.status_code}")

    return result

In [6]:
years = list(range(2020, 2023 + 1))
interfax_df = pd.DataFrame()

for y in years:
    interfax_df_by_year = pd.DataFrame(get_interfax_list_by_year(y))
    interfax_df_by_year['year'] = y
    interfax_df = pd.concat([interfax_df, interfax_df_by_year], ignore_index=True)

interfax_df.to_csv(f"data/interfax_{min(years)}-{max(years)}.csv", index=False)
interfax_df.to_excel(f"data/interfax_{min(years)}-{max(years)}.xlsx", index=False)
interfax_df.head()

,change,description,is_close,point,rank,name,id,url,year
0,0,None,False,1000,1,Московский государственный университет имени М...,1,https://www.msu.ru,2020
1,0,None,False,963,2,Национальный исследовательский ядерный универс...,2,https://mephi.ru,2020
2,0,None,False,961,3,Московский физико-технический институт (национ...,4,https://mipt.ru,2020
3,0,None,False,857,4,Национальный исследовательский университет «Вы...,6,https://www.hse.ru,2020
4,1,None,False,848,5,Новосибирский национальный исследовательский г...,3,http://www.nsu.ru/?lang=ru,2020


В данных рейтинга Интерфакса уже есть данные про движения в рамках рейтинга. Давайте посмотрим на ТОП 10 университетов, которые поднялись в рейтинге за последние года больше всех.

In [7]:
interfax_pivot = pd.pivot_table(
    interfax_df,
    index='name',
    columns='year',
    values='change',
    aggfunc='sum'
)
interfax_pivot['sum'] = interfax_pivot.sum(axis=1)
interfax_pivot.sort_values(by='sum', ascending=False).head(10)

year,2020,2021,2022,2023,sum
name,,,,,
Университет Синергия,-1.0,78.0,43.0,44.0,164.0
Московский институт психоанализа,-8.0,3.0,128.0,25.0,148.0
Чеченский государственный университет имени А.А.Кадырова,-22.0,83.0,58.0,14.0,133.0
Московский государственный психолого-педагогический университет,15.0,78.0,18.0,12.0,123.0
Уральский государственный аграрный университет,-21.0,40.0,104.0,-4.0,119.0
Красноярский государственный аграрный университет,-1.0,-12.0,122.0,3.0,112.0
Уральский государственный горный университет,25.0,17.0,21.0,48.0,111.0
Кировский государственный медицинский университет,25.0,12.0,31.0,40.0,108.0
Кубанский государственный технологический университет,24.0,18.0,61.0,-9.0,94.0
